In [1]:
!pip install requests


In [2]:
import requests as req
from bs4 import BeautifulSoup as bs
from dataclasses import dataclass, asdict
import json
import threading


In [3]:

@dataclass
class Movie:
    name: str
    img: str
    year: str
    length: str
    imdb : str
    release : str
    href: str
    desc: str
    genres: list
    casts: list
    countries: list
    production : list
def crawl(start_page, end_page, movie_url):
    for page_num in range(start_page, end_page+1):
        try:
            new_url = url + f'?page={page_num}'
            soup = bs(req.get(new_url).text, 'html.parser')
            for mv in soup.find_all('div', class_='flw-item'):
                img_tag = mv.find('img')
                name = img_tag['title']
                href = url + mv.find(class_= 'film-name').a['href'].replace('movie/', '')
                img = img_tag['data-src']
                year = mv.find('span', class_='fdi-item').text
                length = mv.find('span', class_='fdi-duration').text
                try:
                    soup2 = bs(req.get(href).text, 'html.parser')
                    desc = soup2.find(class_ = 'description').text.strip()
                    imdb = soup2.find(class_ = 'btn-imdb').text.strip().split(":")[1]
                    row_line = soup2.find(class_ = 'elements').find_all(class_ = 'row-line')
                    release = row_line[0].text.strip().split(":")[1]
                    genres = [i['title'] for i in row_line[1].find_all('a')]
                    casts = [i['title'] for i in row_line[2].find_all('a')]
                    countries = [i['title'] for i in row_line[4].find_all('a')]
                    production = [i['title'] for i in row_line[5].find_all('a')]
                except:
                    print(f'Error at movie href: {href}')
                    continue

                movie = Movie(name, img, year, length , imdb,release, href, desc, genres, casts, countries, production)
                movie_url.append(asdict(movie))
                print(f'{name} done')
        except:
            print(f'Error at page {page_num}')
        print(f'Page {page_num} done', end = '\n\n')
    

In [4]:
num_threads = 2
num_pages = 20
pages_per_thread = num_pages // num_threads
page_ranges = [(i*pages_per_thread+1, (i+1)*pages_per_thread, []) for i in range(num_threads)]
page_ranges[-1] = (page_ranges[-1][0], num_pages, page_ranges[-1][2])
page_ranges

[(1, 10, []), (11, 20, [])]

In [5]:
url = 'https://hdtoday.tv/movie'

threads = []
for page_range in page_ranges:
    start_page, end_page, movie_url = page_range
    thread = threading.Thread(target=crawl, args=(start_page, end_page, movie_url))
    thread.start()
    threads.append(thread)
   
for thread in threads:
    thread.join()

movie_url = []
for page_range in page_ranges:
    movie_url += page_range[2]
        

with open('dummy_data.json', 'a',encoding='utf-8') as f:
    json.dump(movie_url, f, indent=4, ensure_ascii=False)

The Andy Baker Tape done
Spirit - Becoming the World's Best Bartender done
Erin's Guide to Kissing Girls done
Veeran done
Don't Sell My Baby done
Theera Kaadhal done
Death's Roulette done
Taylor Mac's 24-Decade History of Popular Music done
Coffee Wars done
One Ranger done
Sheroes done
Simulant done
Rock Hudson: All That Heaven Allowed done
Lust Stories 2 done
Bobbi Jo: Under the Influence done
Love, Bubbles & Crystal Cove done
Cocoa done
Fireworks done
With The Years That I Have Left done
The Tyrannosaurus Rex done
The Longest Weekend done
Afwaah done
Repeater done
Confidential Informant done
Nimona done
Psycho Sweet 16 done
Anthem done
RSC Live: Troilus and Cressida done
She Is... done
Eldorado: Everything the Nazis Hate done
RSC Live: King John done
Run Rabbit Run done
Past Lives done
My Boss' Wedding done
Hunt done
Love for Starters done
Matt Rife: Only Fans done
How the Holocaust Began done
Love at First Lie done
Maggie Simpson in “Rogue Not Quite One” done
Knights of the Zodiac d

In [6]:
import os

num_cores = os.cpu_count()
print(f'The computer has {num_cores} CPU cores.')

The computer has 4 CPU cores.
